In [ ]:
import requests
# Download .par file
url = "https://raw.githubusercontent.com/pace-neutrons/Horace/master/demo/4to1_124.par"
save_path = "4to1_124.par"
response = requests.get(url, stream=True)
response.raise_for_status()  # Raise an error if the request failed

# Write the file
with open(save_path, "wb") as file:
    for chunk in response.iter_content(chunk_size=8192):
        file.write(chunk)

In [ ]:
from pace_neutrons import Matlab
m = Matlab()

In [ ]:
# import numpy as np

# #Define a dispersion
# # The magnetic form factor of Fe2+
# A=0.0706; a=35.008;  B=0.3589; b=15.358;  C=0.5819; c=5.561;  D=-0.0114;

# # J = 100
# # D_A = 0
# # gam = 30
# # temp = 10
# # amp = 500

# def py_fe_sqw(h, k, l, e, p):
#     js = p[0]
#     d = p[1]
#     om = d + (4*js)*((np.sin(np.pi*h))**2 + (np.sin(np.pi*k))**2 + (np.sin(np.pi*l))**2)
#     q2 = ((1/(2*2.87))**2) * (h**2 + k**2 + l**2)
#     ff = A * np.exp(-a*q2) + B * np.exp(-b*q2) + C * np.exp(-c*q2) + D
#     return amp*(ff**2) * (p[4]/np.pi) * (e / (1-np.exp(-11.602*e/p[3]))) * (4 * p[2] * om) / ((e**2 - om**2)**2 + 4*(p[2] * e)**2)

In [ ]:
import numpy as np

def fm_sqw(h, k, l, e, p):
    """
    Compute the dynamic structure factor S(Q, ω) for Fe using a Damped Harmonic Oscillator (DHO) model.

    Parameters
    ----------
    h, k, l : float or array-like
        Miller indices components.
    e : float or array-like
        Energy transfer (meV).
    p : list or array-like
        Model parameters:
        p[0] = J      (exchange constant)
        p[1] = Delta  (spin gap)
        p[2] = Gamma  (damping)
        p[3] = T      (temperature in K)
        p[4] = scale  (intensity scale factor)

    Returns
    -------
    S : float or array-like
        The scattering function S(Q, ω).
    """
    # Unpack parameters
    J, Delta, Gamma, T, scale = p

    # DHO dispersion relation
    omega = Delta + 4 * J * (
        np.sin(np.pi * h)**2 + 
        np.sin(np.pi * k)**2 + 
        np.sin(np.pi * l)**2
    )

    # Form factor parameters (Fe)
    A, a = 0.0706, 35.008
    B, b = 0.3589, 15.358
    C, c = 0.5819, 5.561
    D = -0.0114

    # Q squared
    q_squared = ((1 / (2 * 2.87))**2) * (h**2 + k**2 + l**2)

    # Magnetic form factor
    ff = A * np.exp(-a * q_squared) + \
         B * np.exp(-b * q_squared) + \
         C * np.exp(-c * q_squared) + D

    # Bose factor (detailed balance)
    bose = e / (1 - np.exp(-11.602 * e / T))

    # Damped Harmonic Oscillator model
    numerator = 4 * Gamma * omega
    denominator = (e**2 - omega**2)**2 + 4 * (Gamma * e)**2
    dho = numerator / denominator

    return scale * (ff**2) * (1 / np.pi) * bose * dho


In [ ]:
import os
def setup_demo_data(m, sqw_filename):
    """
    Internal routine for demo - generates some spe files that can then be
    used in the Horace demo suite.
    """

    J=70
    Delta=0
    Gamma=30
    T=10
    scale=500

    demo_dir = os.getcwd()

    en = list(range(-80, 761, 8))
    par_file = os.path.join(demo_dir, '4to1_124.par')
    sqw_file_single = os.path.join(demo_dir, 'single.sqw')
    efix = 800
    emode = 1
    alatt = [2.87, 2.87, 2.87]
    angdeg = [90, 90, 90]
    u = [1, 0, 0]
    v = [0, 1, 0]
    omega = dpsi = gl = gs = 0

    psi = list(range(0, 91, 4))

    nxspe_limit = len(psi) 
    file_list = [None] * len(psi)

    resulting_sqw_present = os.path.isfile(sqw_filename)

    print('Getting data for Horace demo... Please wait a few minutes')

    try:
        for i in range(len(psi)):
            if i < nxspe_limit:
                file_list[i] = os.path.join(demo_dir, f'HoraceDemoDataFile{i+1}.nxspe')
            else:
                file_list[i] = os.path.join(demo_dir, f'HoraceDemoDataFile{i+1}.spe')

            if os.path.exists(file_list[i]) or resulting_sqw_present:
                continue

            w = m.dummy_sqw(en, par_file, sqw_file_single, efix, emode, alatt, angdeg,
                            u, v, psi[i], omega, dpsi, gl, gs)

            # Make the fake data
            w = m.sqw_eval(w, fm_sqw, [J, Delta, Gamma, T, scale])  # Simulate spinwave cross-section
            w = m.noisify(w, 0.1)  # Add noise to simulate real data

            if i < nxspe_limit:
                d = m.rundatah(w + 0.74)
                m.saveNXSPE(d, file_list[i])
            else:
                d = m.spe(w + 0.74)  # Also add a constant background
                m.save(d, file_list[i])

    except Exception as err:
        if os.path.exists(sqw_file_single):
            os.remove(sqw_file_single)
        print(f'Error producing dummy_sqw data: {err}')
        print('Problem generating data for Horace demo - check that 4to1_124.PAR file is present in the current (demo) directory')

    # Clean up
    if os.path.exists(sqw_file_single):
        os.remove(sqw_file_single)

    return file_list

setup_demo_data(m, 'demo.sqw')

# Creating a SQW file
Now let's create a .sqw file from the generated .nxspe files. We first need to import pace and define the working directory

In [ ]:
import os
demo_dir=os.getcwd()
indir=demo_dir    #% source directory of spe (or nxspe) files
par_file = f"{indir}{os.sep}4to1_124.par"  # detector parameter file
sqw_file = f"{indir}{os.sep}demo.sqw"   # output sqw file
data_source =sqw_file

# Set measurement settings such as incident energy, lattice parameters, etc.
efix = 800
emode = 1
alatt = [2.87, 2.87, 2.87]
angdeg = [90, 90, 90]
u = [1, 0, 0]
v = [0, 1, 0]
omega, dpsi, gl, gs = 0, 0, 0, 0

psi = list(range(0, 91, 4))  # the angle of the sample w.r.t. the incident beam for each run


In [ ]:
# Check if file_list exists
if 'file_list' in locals():
    spe_file=file_list
else:
    spe_file = []
    for i, angle in enumerate(psi):
        file_name = f"{indir}{os.sep}HoraceDemoDataFile{i+1}.nxspe"
        if not os.path.exists(file_name):
            file_name = f"{indir}{os.sep}HoraceDemoDataFile{i+1}.spe"
        spe_file.append(file_name)

# To speed up the calculation, we can use the following commands:
m.hpc('on')
m.hpc_config().parallel_cluster='herbert'
m.hpc_config().combine_sqw_using = 'mpi_code'
m.hpc_config().build_sqw_in_parallel = True

# Generate sqw file
m.gen_sqw(spe_file, par_file, sqw_file, efix, emode, alatt, angdeg, u, v, psi, omega, dpsi, gl, gs)
